#installing dependencies

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1660 Ti
Sat Mar 30 11:56:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     Off | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8              23W / 120W |    334MiB /  6144MiB |     38%      Default |
|                                         |                      |              

This notebook works fine with transformers 4.12, it is not tested on newer versions

Let's download some Arabic text classification datasets

#Creating training datasets

In [2]:
import pandas as pd
import numpy as np
from typing import List
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

This custom dataset class will help us hold our datasets in a structred manner.
It's not necessary to use it with your own data

In [3]:
class CustomDataset:
    def __init__(
        self,
        name: str,
        train: List[pd.DataFrame],
        test: List[pd.DataFrame],
        label_list: List[str],
    ):
        """Class to hold and structure datasets.

        Args:

        name (str): holds the name of the dataset so we can select it later
        train (List[pd.DataFrame]): holds training pandas dataframe with 2 columns ["text","label"]
        test (List[pd.DataFrame]): holds testing pandas dataframe with 2 columns ["text","label"]
        label_list (List[str]): holds the list  of labels
        """
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [4]:
# This will hold all the downloaded and structred datasets
all_datasets= []
DATA_COLUMN = "text"
LABEL_COLUMN = "label"

You can choose which ever dataset you like or use your own.
At this stage we don't do any preprocessing on the text, this is done later when loading the text.

##HARD - Balanced

In [5]:
# df_HARD = pd.read_csv("ArabicTweets.csv", sep="\t", header=0,encoding='windows-1256')

# df_HARD = df_HARD[["text","label"]]  # we are interested in rating and review only
# df_HARD.columns = [DATA_COLUMN, LABEL_COLUMN]
# print(df_HARD[LABEL_COLUMN].value_counts())
# # code rating as +ve if > 3, -ve if less, no 3s in dataset

# hard_map = {
#     1: 'HUMAN',
#     0: 'AI'
# }

# df_HARD[LABEL_COLUMN] = df_HARD[LABEL_COLUMN].apply(lambda x: hard_map[x])
# train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
# label_list_HARD = ['AI', 'HUMAN']

# data_Hard = CustomDataset("HARD", train_HARD, test_HARD, label_list_HARD)
# all_datasets.append(data_Hard)

##ASTD- Unbalanced

In [6]:
df_ASTD_UN = pd.read_csv(
    "ArabicTweets.csv", encoding="windows-1256"
    )

df_ASTD_UN.columns = [DATA_COLUMN, LABEL_COLUMN]

df_ASTD_UN = df_ASTD_UN[df_ASTD_UN[LABEL_COLUMN]!= 'OBJ']

train_ASTD_UN, test_ASTD_UN = train_test_split(
    df_ASTD_UN, test_size=0.2, random_state=42
)

label_list_ASTD_UN = list(df_ASTD_UN[LABEL_COLUMN].unique())
print(label_list_ASTD_UN)
print(df_ASTD_UN[LABEL_COLUMN].value_counts())

data_ASTD_UN = CustomDataset(
    "ASTD-Unbalanced", train_ASTD_UN, test_ASTD_UN, label_list_ASTD_UN
)

all_datasets.append(data_ASTD_UN)

[1, 0]
label
1    157197
0     10028
Name: count, dtype: int64


##ASTD-Dahou-Balanced

In [ ]:
# df_ASTD_B = pd.read_csv(
#     "ArabicTweets.csv",
#     encoding="windows-1256",
# )

# df_ASTD_B.columns = [DATA_COLUMN, LABEL_COLUMN]

# df_ASTD_B[LABEL_COLUMN] = df_ASTD_B[LABEL_COLUMN].apply(lambda x: 'NEG' if (x == -1) else 'POS')

# train_ASTD_B, test_ASTD_B = train_test_split(df_ASTD_B, test_size=0.2, random_state=42)
# label_list_ASTD_B = list(df_ASTD_B[LABEL_COLUMN].unique())

# print(label_list_ASTD_B)
# print(df_ASTD_B[LABEL_COLUMN].value_counts())

# data_ASTD_B = CustomDataset(
#     "ASTD-Dahou-Balanced", train_ASTD_B, test_ASTD_B, label_list_ASTD_B
# )
# all_datasets.append(data_ASTD_B)

##Arsentv-lev

You need to manualy get the ArSenTD-LEV.tsv before running this

In [ ]:
# df_ArSenTD = pd.read_csv(
#     "/content/drive/MyDrive/Datasets/ArSenTD-LEV/ArSenTD-LEV.tsv", sep="\t", header=0
# )

# df_ArSenTD = df_ArSenTD[['Tweet','Sentiment']]

# df_ArSenTD.columns = [DATA_COLUMN, LABEL_COLUMN]

# print(df_ArSenTD[LABEL_COLUMN].value_counts())
# label_list_ArSenTD = list(df_ArSenTD[LABEL_COLUMN].unique())

# train_ArSenTD, test_ArSenTD = train_test_split(
#     df_ArSenTD, test_size=0.2, random_state=42
# )

# data_ArSenTD = CustomDataset("ArSenTD-LEV", train_ArSenTD, test_ArSenTD, label_list_ArSenTD)
# all_datasets.append(data_ArSenTD)

##AJGT

In [ ]:
# df_AJGT = pd.read_excel("/content/drive/My Drive/Datasets/Ajgt/AJGT.xlsx", header=0)

# df_AJGT = df_AJGT[["Feed", "Sentiment"]]
# df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]



# train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.2, random_state=42)

# print(df_AJGT[LABEL_COLUMN].value_counts())
# label_list_AJGT = list(df_AJGT[LABEL_COLUMN].unique())

# data_AJGT = CustomDataset("AJGT", train_AJGT, test_AJGT, label_list_AJGT)
# all_datasets.append(data_AJGT)

##LABR-Unbalanced

In [ ]:
# #@title
# %%writefile labr.py
# # -*- coding: utf-8 -*-
# """
# Created on Sun Mar 10 16:27:03 2013

# @author: Mohamed Aly <mohamed@mohamedaly.info>
# """

# import codecs
# import numpy as np
# import pandas as pd
# import re

# class LABR:
#     def __init__(self):
#         self.REVIEWS_PATH = "LABR/data/"
#         self.RAW_REVIEWS_FILE = "raw_reviews.tsv"
#         self.DELETED_REVIEWS_FILE = "deleted_reviews.tsv"
#         self.CLEAN_REVIEWS_FILE = "reviews.tsv"

#     # Copied from the PyArabic package.
#     def arabicrange(self):
#         """return a list of arabic characteres .
#         Return a list of characteres between \u060c to \u0652
#         @return: list of arabic characteres.
#         @rtype: unicode;
#         """
#         mylist=[];
#         for i in range(0x0600, 0x00653):
#             try :
#                 mylist.append(unichr(i));
#             except ValueError:
#                 pass;
#         return mylist;

#     # cleans a single review
#     def clean_raw_review(self, body):
#          # patterns to remove first
#         pat = [\
#             (u'http[s]?://[a-zA-Z0-9_\-./~\?=%&]+', u''),               # remove links
#             (u'www[a-zA-Z0-9_\-?=%&/.~]+', u''),
# #            u'\n+': u' ',                     # remove newlines
#             (u'<br />', u' '),                  # remove html line breaks
#             (u'</?[^>]+>', u' '),              # remove html markup
# #            u'http': u'',
#             (u'[a-zA-Z]+\.org', u''),
#             (u'[a-zA-Z]+\.com', u''),
#             (u'://', u''),
#             (u'&[^;]+;', u' '),
#             (u':D', u':)'),
# #            (u'[0-9/]+', u''),
# #            u'[a-zA-Z.]+': u'',
# #            u'[^0-9' + u''.join(self.arabicrange()) + \
# #                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
# #                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
#             (u'\s+', u' '),                     # remove spaces
#             (u'\.+', u'.'),                     # multiple dots
#             (u'[\u201C\u201D]', u'"'),          #“
#             (u'[\u2665\u2764]', u''),           # heart symbol
#             (u'[\u00BB\u00AB]', u'"'),
#             (u'\u2013', u'-'),                # dash
#         ]

#         # patterns that disqualify a review
#         remove_if_there = [\
#             (u'[^0-9' + u''.join(self.arabicrange()) + \
#                 u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
#                 u'\s\^><\-_\u201D\u00AB=\u2026+|' + \
#                 u'\u0660-\u066D\u201C\u201D' + \
#                 u'\ufefb\ufef7\ufef5\ufef9]+', u''),                   # non arabic characters
#         ]

#         # patterns that disqualify if empty after removing
#         remove_if_empty_after = [\
#             (u'[0-9a-zA-Z\-_]', u' '),             #alpha-numeric
#             (u'[0-9' + u".,!;:$%&*%'#(){}~`\[\]/\\\\\"" + \
#                 u'\s\^><`\-=_+]+', u''),                  # remove just punctuation
#             (u'\s+', u' '),                     # remove spaces
#         ]

#         # remove again
#         # patterns to remove
#         pat2 = [\
# #            u'[^0-9' + u''.join(self.arabicrange()) + \
# #                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
# #                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
#         ]

#         skip = False

#         # if empty body, skip
#         if body == u'': skip = True

#         # do some subsitutions
#         for k,v in pat:
#             body = re.sub(k, v, body)

#         # remove if exist
#         for k,v in remove_if_there:
#             if re.search(k, body):
#                 skip = True

#         # remove if empty after replacing
#         for k,v in remove_if_empty_after:
#             temp = re.sub(k, v, body)
#             if temp == u" " or temp == u"":
#                 skip = True

#         # do some more subsitutions
#         if not skip:
#             for k,v in pat2:
#                 body = re.sub(k, v, body)

#         # if empty string, skip
#         if body == u'' or body == u' ':
#             skip = True

#         if not skip:
#             return body
#         else:
#             return u""

#     # Read raw reviews from file and clean and write into clean_reviews
#     def clean_raw_reviews(self):
#         # input file
#         in_file = codecs.open(self.REVIEWS_PATH + self.RAW_REVIEWS_FILE,
#                               'r', encoding="utf-8")
#         reviews = in_file.readlines()

#         # Output file: rating<tab>content
#         out_file = open(self.REVIEWS_PATH + self.CLEAN_REVIEWS_FILE,
#                         'w', buffering = 100)
#         deleted_file = open(self.REVIEWS_PATH + self.DELETED_REVIEWS_FILE,
#                             'w', buffering = 100)

#         counter = 1
#         for i in xrange(0, len(reviews)):
#             review = reviews[i]
#             skip = False

# #           # If line starts with #, then skip
# #            if review[0] == u"#": continue

#             # split by <tab>
#             parts = review.split(u"\t")

#             # rating is first part and body is last part
#             rating = parts[0]
#             review_id = parts[1]
#             user_id = parts[2]
#             book_id = parts[3]
#             body = parts[4].strip()

#             # clean body
#             body = self.clean_raw_review(body)
#             if body == u"": skip = True

#             if i % 5000 == 0:
#                 print("review %d:" % (i))

#             # write output
#             line = u"%s\t%s\t%s\t%s\t%s\n" % (rating, review_id, user_id,
#                                               book_id, body)
#             if not skip:
#                 out_file.write(line.encode('utf-8'))
#                 counter += 1
#             else:
#                 deleted_file.write(line.encode('utf-8'))

#     # Read the reviews file. Returns a tuple containing these lists:
#     #   rating: the rating 1 -> 5
#     #   review_id: the id of the review
#     #   user_id: the id of the user
#     #   book_id: the id of the book
#     #   body: the text of the review
#     def read_review_file(self, file_name):
#         reviews = codecs.open(file_name, 'r', 'utf-8').readlines()

#         # remove comment lines and newlines
#         reviews = [r.strip() for r in reviews if r[0] != u'#']

#         # parse
#         rating = list()
#         review_id = list()
#         user_id = list()
#         book_id = list()
#         body = list()
#         for review in reviews:
#             # split by <tab>
#             parts = review.split(u"\t")

#             # rating is first part and body is last part
#             rating.append(int(parts[0]))
#             review_id.append(parts[1])
#             user_id.append(parts[2])
#             book_id.append(parts[3])
#             if len(parts) > 4:
#                 body.append(parts[4])
#             else:
#                 body.append(u"")

#         return (rating, review_id, user_id, book_id, body)

#     # Writes reviews to a file
#     def write_review_file(self, file_name, rating, review_id, user_id,
#                           book_id, body):

#         lines = list()
#         # loop
#         for i in xrange(len(rating)):
#             line = u"%s\t%s\t%s\t%s\t%s\n" % (rating[i], review_id[i],
#                                               user_id[i], book_id[i],
#                                               body[i])
#             lines.append(line)

#         open(file_name, 'w').write(u''.join(lines).encode('utf-8'))

#     def read_clean_reviews(self):
#         return self.read_review_file(self.REVIEWS_PATH +
#                                      self.CLEAN_REVIEWS_FILE)

#     def read_raw_reviews(self):
#         return self.read_review_file(self.REVIEWS_PATH + self.RAW_REVIEWS_FILE)

#     # Splits the dataset into a training/test sets in the setting of using 5
#     # classes (predicting the rating value from 1 to 5)
#     def split_train_test_5class(self, rating, percent_test,
#                                 balanced = "unbalanced"):
#         np.random.seed(1234)

#         num_reviews = len(rating)
#         review_ids = np.arange(0, num_reviews)

#         if balanced == "unbalanced":
#             ntest = np.floor(num_reviews * percent_test)
#             np.random.shuffle(review_ids)

#             test_ids = review_ids[:ntest]
#             train_ids = review_ids[ntest:]

#         elif balanced == "balanced":
#             (sizes, bins) = np.histogram(rating, [1, 2, 3, 4, 5, 6])
#             min_size = np.min(sizes)
#             print(min_size)

#             # sample review ids equally among classes
#             test_ids = np.zeros((0,), dtype="int32")
#             train_ids = np.zeros((0,), dtype="int32")
#             rating = np.array(rating)
#             ntest = np.floor(min_size * percent_test)
#             for c in range(1, 6):
#                 cids = review_ids[np.nonzero(rating == c)]
#                 np.random.shuffle(cids)

#                 test_ids = np.r_[test_ids, cids[:ntest]]
#                 train_ids = np.r_[train_ids, cids[ntest:min_size]]

#         train_file = self.REVIEWS_PATH + "5class-" + balanced+ "-train.txt"
#         test_file = self.REVIEWS_PATH + "5class-" + balanced+ "-test.txt"

#         open(train_file, 'w').write('\n'.join(map(str, train_ids)))
#         open(test_file, 'w').write('\n'.join(map(str, test_ids)))

#         return (train_ids, test_ids)

#     # Splits the dataset into a training/test sets in the setting of using 2
#     # classes (predicting the polarity of the review where ratings 1 & 2
#     # are considered negative, ratings 4 & 5 are positive, and rating 3 is
#     # ignored)
#     def split_train_test_2class(self, rating, percent_test,
#                                 balanced = "unbalanced"):
#         np.random.seed(1234)

#         rating = np.array(rating, dtype='int32')
#         # length
#         num_reviews = len(rating)
#         review_ids = np.arange(0, num_reviews)

#         # convert to binary, with ratings [1, 2] --> neg and [4, 5] --> pos
#         rating[rating == 2] = 1
#         rating[rating == 4] = 5

#         ids = (rating == 1) + (rating == 5)
#         review_ids = review_ids[ids]
#         rating = rating[ids]
#         rating[rating == 1] = 0
#         rating[rating == 5] = 1

#         # get length after filtering
#         num_reviews = rating.shape[0]

#         if balanced == "unbalanced":
#             ntest = np.floor(num_reviews * percent_test)
#             np.random.shuffle(review_ids)

#             test_ids = review_ids[:ntest]
#             train_ids = review_ids[ntest:]

#         elif balanced == "balanced":
#             (sizes, bins) = np.histogram(rating, [0, 1, 2])
#             min_size = np.min(sizes)
#             print(min_size)

#             # sample review ids equally among classes
#             test_ids = np.zeros((0,), dtype="int32")
#             train_ids = np.zeros((0,), dtype="int32")
#             rating = np.array(rating)
#             ntest = np.floor(min_size * percent_test)
#             for c in [0, 1]:
#                 cids = review_ids[np.nonzero(rating == c)]
#                 np.random.shuffle(cids)

#                 test_ids = np.r_[test_ids, cids[:ntest]]
#                 train_ids = np.r_[train_ids, cids[ntest:min_size]]

#         train_file = self.REVIEWS_PATH + "2class-" + balanced+ "-train.txt"
#         test_file = self.REVIEWS_PATH + "2class-" + balanced+ "-test.txt"

#         open(train_file, 'w').write('\n'.join(map(str, train_ids)))
#         open(test_file, 'w').write('\n'.join(map(str, test_ids)))

#         return (train_ids, test_ids)

#     # Reads a training or test file. The file contains the indices of the
#     # reviews from the clean reviews file.
#     def read_train_test_file(self, file_name):
#         ins = open(file_name).readlines()
#         ins = [int(i.strip()) for i in ins]

#         return ins

#     # A helpter function.
#     def set_binary_klass(self, ar):
#         ar[(ar == 1) + (ar == 2)] = 0
#         ar[(ar == 4) + (ar == 5)] = 1

#     # Returns (train_x, train_y, test_x, test_y)
#     # where x is the review body and y is the rating (1->5 or 0->1)
#     def get_train_test(self, klass = "2", balanced = "balanced"):
#         (rating, a, b, c, body) = self.read_clean_reviews()
#         rating = np.array(rating)
#         body = pd.Series(body)

#         train_file = (self.REVIEWS_PATH + klass + "class-" +
#             balanced+ "-train.txt")
#         test_file = (self.REVIEWS_PATH + klass + "class-" +
#             balanced+ "-test.txt")

#         train_ids = self.read_train_test_file(train_file)
#         test_ids = self.read_train_test_file(test_file)

#         train_y = rating[train_ids]
#         test_y = rating[test_ids]
#         train_x = body[train_ids]
#         test_x = body[test_ids]

#         if klass == "2":
#             self.set_binary_klass(train_y)
#             self.set_binary_klass(test_y)

#         return (train_x, train_y, test_x, test_y)


In [ ]:
# from labr import LABR

# labr_helper = LABR()

# (d_train, y_train, d_test, y_test) = labr_helper.get_train_test(
#     klass="2", balanced="unbalanced"
# )

# train_LABR_B_U = pd.DataFrame({DATA_COLUMN: d_train, LABEL_COLUMN: y_train})
# test_LABR_B_U = pd.DataFrame({DATA_COLUMN: d_test, LABEL_COLUMN: y_test})

# train_LABR_B_U[LABEL_COLUMN] = train_LABR_B_U[LABEL_COLUMN].apply(lambda x: 'NEG' if (x == 0) else 'POS')
# test_LABR_B_U[LABEL_COLUMN] = test_LABR_B_U[LABEL_COLUMN].apply(lambda x: 'NEG' if (x == 0) else 'POS')

# print(train_LABR_B_U[LABEL_COLUMN].value_counts() + test_LABR_B_U[LABEL_COLUMN].value_counts())
# label_list_LABR_B_U = list(test_LABR_B_U[LABEL_COLUMN].unique())

# data_LABR_B_U = CustomDataset(
#     "LABR-UN-Binary", train_LABR_B_U, test_LABR_B_U, label_list_LABR_B_U
# )
# all_datasets.append(data_LABR_B_U)

In [ ]:
# for x in all_datasets:
#   print(x.name)

## ArSAS

In [ ]:
# df_ArSAS = pd.read_csv("/content/ArSAS..txt", sep="\t",encoding='utf-8')
# df_ArSAS = df_ArSAS[["Tweet_text","Sentiment_label"]]  # we are interested in rating and review only
# df_ArSAS.columns = [DATA_COLUMN, LABEL_COLUMN]
# print("Total length: ", len(df_ArSAS))
# print(df_ArSAS[LABEL_COLUMN].value_counts())

# label_list_ArSAS = list(df_ArSAS[LABEL_COLUMN].unique())
# print(label_list_ArSAS)

# train_ArSAS, test_ArSAS = train_test_split(df_ArSAS, test_size=0.2, random_state=42)
# print("Training length: ", len(train_ArSAS))
# print("Testing length: ", len(test_ArSAS))
# data_ArSAS = CustomDataset("ArSAS", train_ArSAS, test_ArSAS, label_list_ArSAS)
# all_datasets.append(data_ArSAS)

#Trainer

Start the training procedure

In [ ]:
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import copy

from arabert.preprocess import ArabertPreprocessor
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures

List all the datasets we have

In [ ]:
for x in all_datasets:
  print(x.name)

In [ ]:
# select a dataset
dataset_name = 'ArSAS'
# select a model from the huggingface modelhub https://huggingface.co/models?language=ar
model_name = 'aubmindlab/bert-base-arabertv02-twitter' # we are going to use the twitter AraBERT since it has emojis and dialects

In [ ]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = copy.deepcopy(d)
    print('Dataset found')
    break

Create and apply preprocessing using the AraBERT processor

In [ ]:
arabic_prep = ArabertPreprocessor(model_name)

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x: arabic_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x: arabic_prep.preprocess(x))

In [ ]:
# Sanity check on the dataset
list(selected_dataset.train[DATA_COLUMN][0:10])

Now we need to check the tokenized sentence length to decide on the maximum sentence length value

In [ ]:
tok = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print("Training Sentence Lengths: ")
plt.hist([ len(tok.tokenize(sentence)) for sentence in selected_dataset.train[DATA_COLUMN].to_list()],bins=range(0,128,2))
plt.show()

print("Testing Sentence Lengths: ")
plt.hist([ len(tok.tokenize(sentence)) for sentence in selected_dataset.test[DATA_COLUMN].to_list()],bins=range(0,128,2))
plt.show()

Let's select 100 as our maximum sentence length, and check how many sequences will be truncated

In [ ]:
max_len = 128

In [ ]:
print("Truncated training sequences: ", sum([len(tok.tokenize(sentence)) > max_len for sentence in selected_dataset.test[DATA_COLUMN].to_list()]))

print("Truncated testing sequences: ", sum([len(tok.tokenize(sentence)) > max_len for sentence in selected_dataset.test[DATA_COLUMN].to_list()]))

8 out of ~4000 for testing isn't bad

Now let's create a classification dataset to load the data

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()
      """
      Args:
      text (List[str]): List of the training text
      target (List[str]): List of the training labels
      tokenizer_name (str): The tokenizer name (same as model_name).
      max_len (int): Maximum sentence length
      label_map (Dict[str,int]): A dictionary that maps the class labels to integer
      """
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map


    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
      )
      return InputFeatures(**inputs,label=self.label_map[self.target[item]])

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)

train_dataset = ClassificationDataset(
    selected_dataset.train[DATA_COLUMN].to_list(),
    selected_dataset.train[LABEL_COLUMN].to_list(),
    model_name,
    max_len,
    label_map
  )
test_dataset = ClassificationDataset(
    selected_dataset.test[DATA_COLUMN].to_list(),
    selected_dataset.test[LABEL_COLUMN].to_list(),
    model_name,
    max_len,
    label_map
  )

Check the dataset output

In [ ]:
print(next(iter(train_dataset)))

Create a function that return a pretrained model ready to do classification

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

Define whatever metric you want here

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  #macro_precision = precision_score(p.label_ids,preds,average='macro')
  #macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'accuracy': acc
  }

In [ ]:
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark = False

#Regular Training

Define our training parameters.
Check the TrainingArguments documentation for more options https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments

In [ ]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 128,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 2,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    greater_is_better = True,
    seed = 25
  )

set_seed(training_args.seed)

Create the trainer

In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
#start the training
trainer.train()

Save the model, the tokenizer and the config

In [ ]:
inv_label_map = inv_label_map = { v:k for k, v in label_map.items()}
print(inv_label_map)
trainer.model.config.label2id = label_map
trainer.model.config.id2label = inv_label_map
trainer.save_model("output_dir")
train_dataset.tokenizer.save_pretrained("output_dir")

In [ ]:
#copy the model to drive
!cp output_dir /content/drive/MyDrive

## predict using the saved model

In [ ]:
from transformers import pipeline

In [ ]:
# initialize pipline
pipe = pipeline("sentiment-analysis", model="output_dir", device=0, return_all_scores=True)

In [ ]:
pipe("Some Text")

# K-fold

This section is bit more advanced.

We will divide the training set into K-folds and train model with cross-validation to check for the best hyper-parameters before check the performance on the test set.

Alternatively, you can combine the training and testing set if you are participating in a competition, then ensemble the output models

In [ ]:
# do kfold on the training. Check the perfomance on the test set
kfold_dataset = selected_dataset.train
# do kfold on all the dataset. Here we will not have any dataset to checl final performance on (this is used mainly in competitions)
# kfold_dataset = pd.concat([selected_dataset.train,selected_dataset.test])
kfold_dataset.reset_index(inplace=True,drop=True)

In [ ]:
# this is used later
inv_label_map = { v:k for k, v in label_map.items()}

Defing the number of Stratified kfold splits

In [ ]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=123
  )

Train using cross validation and save the best model at each fold

In [ ]:
all_results = []
fold_best_f1 = 0
best_fold = None
for fold_num , (train, dev) in enumerate(kf.split(kfold_dataset,kfold_dataset['label'])):
  print("**************************Starting Fold Num: ", fold_num," **************************")

  train_dataset = ClassificationDataset(list(kfold_dataset[DATA_COLUMN][train]),
                              list(kfold_dataset[LABEL_COLUMN][train]),
                              model_name,
                              max_len,
                              label_map)

  val_dataset = ClassificationDataset(list(kfold_dataset[DATA_COLUMN][dev]),
                              list(kfold_dataset[LABEL_COLUMN][dev]),
                              model_name,
                              max_len,
                              label_map)

  training_args = TrainingArguments(
    output_dir= f"./train_{fold_num}",
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 128,
    gradient_accumulation_steps = 2,
    num_train_epochs= 2,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'macro_f1',
    greater_is_better = True,
    seed = 123
  )

  set_seed(training_args.seed)

  trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
  )
  trainer.model.config.label2id = label_map
  trainer.model.config.id2label = inv_label_map

  trainer.train()

  results = trainer.evaluate()
  all_results.append(results)
  print(results)

  trainer.save_model(f"./train_{fold_num}/best_model")
  val_dataset.tokenizer.save_pretrained(f"./train_{fold_num}/best_model")

  # delete the rest of the checkpoints
  !rm -rf f"./train_{fold_num}/checkpoint-*"

  if results['eval_macro_f1'] > fold_best_f1:
    print('**************************New Best Model Found!**************************')
    fold_best_f1 = results['eval_macro_f1']
    best_fold = fold_num

In [ ]:
all_results

In [ ]:
from statistics import mean
mean([x['eval_macro_f1'] for x in all_results])

After checking for the best hyper parameters you should use the regular training section and retrain the model with the parameters that you had here.

Or Ensemble the models together.

## Ensemble all the cross validation models

In [ ]:
from transformers import pipeline
import more_itertools

In [ ]:
inv_label_map = { v:k for k, v in label_map.items()}

Load some file which has text that we need to run inference on.
I will use the test set for that

In [ ]:
# pred_df = prediction['Text']
# pred_df = pred_df.apply(lambda x:   arabic_prep.preprocess(x))

pred_df = selected_dataset.test[DATA_COLUMN]

In [ ]:
cross_val_df = pd.DataFrame([])
for i in range(0,5):
  pipe = pipeline("sentiment-analysis", model=f"train_{i}/best_model", device=0, return_all_scores =True, max_length=max_len, truncation=True)
  preds = []
  for s in tqdm(more_itertools.chunked(list(pred_df), 32)): # batching for faster inference
    preds.extend(pipe(s))
  cross_val_df[f'model_{i}'] = preds

In [ ]:
from collections import defaultdict

final_labels = []
final_scores = []
for id, row in cross_val_df.iterrows():
  total_score = defaultdict(lambda: 0)
  for pred in row:
    for cls in pred:
      total_score[cls['label']] += cls['score']

  avg_score = { k: v/ 5 for k, v in total_score.items()}

  final_labels.append(max(avg_score, key=avg_score.get))
  final_scores.append(avg_score[max(avg_score, key=avg_score.get)])

In [ ]:
cross_val_df['preds'] = final_labels
cross_val_df['sentiment_score'] = final_scores

In [ ]:
cross_val_df['preds'].value_counts()

In [ ]:
print(classification_report(selected_dataset.test[LABEL_COLUMN],cross_val_df['preds']))